In [ ]:
pip install python-dotenv


In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import re
import os
from dotenv import load_dotenv

In [13]:
from google.colab import files
uploaded = files.upload()  # Use file picker to upload your local `.env` file


Saving .env to .env (1)


In [15]:
load_dotenv(".env")

HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    print(" Token loaded successfully!")
else:
    print("Token not loaded. Check file content.")


 Token loaded successfully!


In [16]:
# Login securely
login(HF_TOKEN)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# Loading the gated Mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", use_auth_token=True)

# Setting up generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)

# Safety filters
def is_safe_query(query):
    unsafe_patterns = [
        r"suicide", r"overdose", r"\bkill\b", r"emergency", r"bleeding", r"unconscious",
        r"take without prescription", r"replace doctor", r"stop medication suddenly"
    ]
    return not any(re.search(p, query.lower()) for p in unsafe_patterns)

# Funtion for Building the prompt
def build_prompt(user_query):
    return f"""[INST] You are a helpful and friendly medical assistant chatbot.
Answer general health-related questions clearly, simply, and cautiously.
Do not provide medical diagnoses or personalized treatment.
Always encourage users to consult a qualified doctor when appropriate.

User: {user_query}
Response: [/INST]"""

# Chatbot function
def ask_healthbot(user_query):
    if not is_safe_query(user_query):
        return "I'm sorry, but I can't help with that request. Please consult a healthcare professional immediately."

    prompt = build_prompt(user_query)

    try:
        response = generator(prompt, do_sample=True, temperature=0.7)[0]["generated_text"]
        if "[/INST]" in response:
            return response.split("[/INST]")[-1].strip()
        return response.strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"

# --- Chatbot loop ---
if __name__ == "__main__":
    print("🤖 HealthBot (Mistral): Ask a health question. Type 'exit' to quit.")
    while True:
        query = input("\nAsk a health question: ")
        if query.lower() in {"exit", "quit"}:
            print("Goodbye!")
            break
        answer = ask_healthbot(query)
        print("\n🤖", answer)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...


🤖 Diabetes is a medical condition where there is an increase in blood sugar levels. The symptoms can vary from person to person, but some common symptoms include:

1. Frequent urination: This is because the kidneys are working harder to filter excess sugar from the blood, which can lead to more frequent urination.
2. Increased thirst: As the body loses fluids from frequent urination, you may feel more thirsty than usual.
3. Fatigue and weakness: High blood sugar levels can cause fatigue and a lack of energy.
4. Blurry vision: High blood sugar levels can cause the lens of the eye to swell, which can lead to blurry vision. If left untreated, this could potentially lead to more serious eye conditions.
5. Slow healing: High blood sugar levels can weaken the immune system, making it harder for wounds to heal.
6. Increased appetite: High blood sugar levels can cause increased hunger, which can lead to weight gain.

If you are experiencing any of these symptoms, it's important to consult a q

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 A normal blood pressure range is generally considered to be between 90/60 mmHg and 120/80 mmHg. However, these ranges can vary slightly depending on your age, gender, and other factors. It's always a good idea to consult with a healthcare provider to determine what a healthy blood pressure range is for you.

Ask a health question: How much water should I drink per day?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 The recommended daily water intake varies depending on factors such as age, weight, and activity level. For adults, the general guideline is to drink at least 8 glasses of water per day or around 2 liters. However, it's always best to consult with your healthcare provider for personalized advice based on your individual needs.

Ask a health question: exit
Goodbye!
